In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

import savitzky_golay as sg

from Fitting import double_logistic as dl
from Fitting import Cubic as cb

from Fitting import twoderive as dv
from Fitting import ratio as ra
from Fitting import seasonal_amplitude as samp

#%%
def read_data(yr):
    inpath = (r'D:/Gosif_CHN/CHN/sif_CHN_%d.nc' %yr)
    global lat, lon
    with nc.Dataset(inpath) as f:
        sif=(f.variables['sif'][:])
        lat=(f.variables['lat'][:])
        lon=(f.variables['lon'][:])
        
    sif=sif*0.0001    
    return sif

### method1 sg+dbl+derive(一阶导+二阶导)

In [2]:
def sg_dbl(sif):
    doys=np.arange(1, 365, 8)
    tt=np.arange(1, 366, 1)
    
    SOP1=np.zeros((600, 1201))
    EOP1=np.zeros((600, 1201))
    GSL1=np.zeros((600, 1201))
    
    POP=np.zeros((600, 1201))
    
    SOP2=np.zeros((600, 1201))
    EOP2=np.zeros((600, 1201))
    GSL2=np.zeros((600, 1201))
    
    for r in range(600):
        if r%30==0:
            print('still alive!')
        for c in range(1201):
            if sif[:, r, c].mean()<0 or sif[:, r, c].mean()>3:
                SOP1[r, c]=-9999
                EOP1[r, c]=-9999
                GSL1[r, c]=-9999
                
                POP[r, c]=-9999
                
                SOP2[r, c]=-9999
                EOP2[r, c]=-9999
                GSL2[r, c]=-9999
            else:
                sif_sg=sg.savitzky_golay(sif[:, r, c], 15, 4)
                sif_dbl=dl.fit_phenology_model_double_logistic(sif_sg, doys)
                
                SOP1[r, c], EOP1[r, c]=dv.derive(sif_dbl, tt)
                GSL1[r, c]=EOP1[r, c]-SOP1[r, c]
                
                POP[r, c]=sif_dbl.argmax()
                
                SOP2[r, c], EOP2[r, c]=dv.derive2(sif_dbl, tt)
                GSL2[r, c]=EOP2[r, c]-SOP2[r, c]
                
    return SOP1, EOP1, GSL1, SOP2, EOP2, GSL2, POP

#%%
'''生成新的nc文件'''
def CreatNC(data1, data2, data3, data4, data5, data6, data7, yr):
    
    new_NC = nc.Dataset(r"D:/Gosif_CHN/CHN/CHN_dbl_derive_%d.nc"%yr , 'w', format='NETCDF4')
    
    new_NC.createDimension('lat', 600)
    new_NC.createDimension('lon', 1201)
    
    var=new_NC.createVariable('SOP1', 'f', ("lat","lon"))
    var=new_NC.createVariable('EOP1', 'f', ("lat","lon"))
    var=new_NC.createVariable('GSL1', 'f', ("lat","lon"))
    var=new_NC.createVariable('SOP2', 'f', ("lat","lon"))
    var=new_NC.createVariable('EOP2', 'f', ("lat","lon"))
    var=new_NC.createVariable('GSL2', 'f', ("lat","lon"))
    var=new_NC.createVariable('POP', 'f', ("lat","lon"))
    new_NC.createVariable('lat', 'f', ("lat"))
    new_NC.createVariable('lon', 'f', ("lon"))
    
    new_NC.variables['SOP1'][:]=data1
    new_NC.variables['EOP1'][:]=data2
    new_NC.variables['GSL1'][:]=data3
    new_NC.variables['SOP2'][:]=data4
    new_NC.variables['EOP2'][:]=data5
    new_NC.variables['GSL2'][:]=data6
    new_NC.variables['POP'][:]=data7
    new_NC.variables['lat'][:]=lat
    new_NC.variables['lon'][:]=lon
        
    
    #最后记得关闭文件
    new_NC.close()  



### method2+method3

In [3]:
#%%cubic+03amplitude
print('-------------------------cubic+03amplitude--------------------')
def sg_cb_samp(sif):
    doys=np.arange(1, 365, 8)
    t=np.arange(1, 362, 1)
    
    SOP=np.zeros((800, 1400))
    EOP=np.zeros((800, 1400))
    GSL=np.zeros((800, 1400))
    threshold=np.zeros((800, 1400))
    
    for r in range(800):
        if r%30==0:
            print('still alive!')
        for c in range(1400):
            if sif[:, r, c].mean()<0 or sif[:, r, c].mean()>3:
                SOP[r, c]=-9999
                EOP[r, c]=-9999
                GSL[r, c]=-9999
                
            else:
                sif_sg=sg.savitzky_golay(sif[:, r, c], 15, 4)
                sif_cb=cb.cubic(doys, sif_sg)
                
                SOP[r, c], EOP[r, c], threshold[r, c]=samp.amp03(sif_cb)
                GSL[r, c] = EOP[r, c]-SOP[r, c]
    return SOP, EOP, GSL

#%%cubic+ratio2
print('-------------------------cubic+ratio2--------------------')
def sg_cb_ra2(sif):
    doys=np.arange(1, 365, 8)
    t=np.arange(1, 362, 1)
    
    SOP=np.zeros((800, 1400))
    EOP=np.zeros((800, 1400))
    GSL=np.zeros((800, 1400))
    threshold=np.zeros((800, 1400))
    
    for r in range(800):
        if r%30==0:
            print('still alive!')
        for c in range(1400):
            if sif[:, r, c].mean()<0 or sif[:, r, c].mean()>3:
                SOP[r, c]=-9999
                EOP[r, c]=-9999
                GSL[r, c]=-9999
                
            else:
                sif_sg=sg.savitzky_golay(sif[:, r, c], 15, 4)
                sif_cb=cb.cubic(doys, sif_sg)
                
                SOP[r, c], EOP[r, c]= ra.ratio2(sif_cb)
                GSL[r, c] = EOP[r, c]-SOP[r, c]
    return SOP1, EOP1, GSL1, SOP2, EOP2, GSL2

#%%
'''生成新的nc文件'''
def CreatNC2(data1, data2, data3, yr):
    
    new_NC = nc.Dataset(r"D:/Gosif_CHN/phenology/CHN_cb_samp_%d.nc"%yr , 'w', format='NETCDF4')
    
    new_NC.createDimension('lat', 800)
    new_NC.createDimension('lon', 1400)
    
    var=new_NC.createVariable('SOP', 'f', ("lat","lon"))
    var=new_NC.createVariable('EOP', 'f', ("lat","lon"))
    var=new_NC.createVariable('GSL', 'f', ("lat","lon"))
    new_NC.createVariable('lat', 'f', ("lat"))
    new_NC.createVariable('lon', 'f', ("lon"))
    
    new_NC.variables['SOP'][:]=data1
    new_NC.variables['EOP'][:]=data2
    new_NC.variables['GSL'][:]=data3
    new_NC.variables['lat'][:]=lat
    new_NC.variables['lon'][:]=lon
        
    
    #最后记得关闭文件
    new_NC.close()  


-------------------------cubic+03amplitude--------------------
-------------------------cubic+ratio2--------------------


### method2和method3合并为一个文件

In [2]:
print('-------------------------cubic+ratio2+03amplitude--------------------')
def sg_cb_all(sif):
    doys = np.arange(1, 365, 8)
    t = np.arange(1, 362, 1)

    SOP1=np.zeros((800, 1400))
    EOP1=np.zeros((800, 1400))
    GSL1=np.zeros((800, 1400))
    
    
    SOP2=np.zeros((800, 1400))
    EOP2=np.zeros((800, 1400))
    GSL2=np.zeros((800, 1400))

    for r in range(800):
        if r % 30 == 0:
            print('still alive!')
        for c in range(1400):
            if sif[:, r, c].mean() < 0 or sif[:, r, c].mean() > 3:
                SOP1[r, c] = -9999
                EOP1[r, c] = -9999
                GSL1[r, c] = -9999
                
                SOP2[r, c] = -9999
                EOP2[r, c] = -9999
                GSL2[r, c] = -9999

            else:
                sif_sg = sg.savitzky_golay(sif[:, r, c], 15, 4)
                sif_cb = cb.cubic(doys, sif_sg)
                
                SOP1[r, c], EOP1[r, c]= ra.ratio2(sif_cb)
                GSL1[r, c] = EOP1[r, c]-SOP1[r, c]

                SOP2[r, c], EOP2[r, c], threshold = samp.amp03(sif_cb)
                GSL2[r, c] = EOP2[r, c]-SOP2[r, c]
    return SOP1, EOP1, GSL1, SOP2, EOP2, GSL2

'''生成新的nc文件'''
def CreatNC3(data1, data2, data3, data4, data5, data6, yr):
    
    new_NC = nc.Dataset(r"D:/Gosif_CHN/phenology_new/CHN_cubic_%d.nc"%yr , 'w', format='NETCDF4')
    
    new_NC.createDimension('lat', 800)
    new_NC.createDimension('lon', 1400)
    
    var=new_NC.createVariable('SOP1', 'f', ("lat","lon"))
    new_NC.createVariable('EOP1', 'f', ("lat","lon"))
    new_NC.createVariable('GSL1', 'f', ("lat","lon"))
    new_NC.createVariable('SOP2', 'f', ("lat","lon"))
    new_NC.createVariable('EOP2', 'f', ("lat","lon"))
    new_NC.createVariable('GSL2', 'f', ("lat","lon"))
    new_NC.createVariable('lat', 'f', ("lat"))
    new_NC.createVariable('lon', 'f', ("lon"))
    
    new_NC.variables['SOP1'][:]=data1
    new_NC.variables['EOP1'][:]=data2
    new_NC.variables['GSL1'][:]=data3
    new_NC.variables['SOP2'][:]=data4
    new_NC.variables['EOP2'][:]=data5
    new_NC.variables['GSL2'][:]=data6
    new_NC.variables['lat'][:]=lat
    new_NC.variables['lon'][:]=lon
    
    var.description= ("SOP1是method2 ratio2, SOP2是method3 seasonal amplitude")
    
    #最后记得关闭文件
    new_NC.close()  

-------------------------cubic+ratio2+03amplitude--------------------


In [ ]:
#%%sg+dbl+derive
if __name__=='__main__':
    for yr in range(2019,2022):
        sif=read_data(yr)
        sop1, eop1, gsl1, sop2, eop2, gsl2, pop = sg_dbl(sif)
        CreatNC(sop1, eop1, gsl1, sop2, eop2, gsl2, pop, yr)
        print("%d phenology is done" %yr)

In [7]:
#%%sg+cubic+seasonal_amplitude
if __name__=='__main__':
    for yr in range(2009,2022):
        sif=read_data(yr)
        sop, eop, gsl = sg_cb_samp(sif)
        CreatNC2(sop, eop, gsl, yr)
        print("%d phenology is done" %yr)

still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
2009 phenology is done
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
2010 phenology is done
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still

In [3]:
#%%sg+cubic+ratio2
if __name__=='__main__':
    for yr in range(2007,2008):
        sif=read_data(yr)
        sos, eos, gsl = sg_cb_ra2(sif)
        CreatNC2(sos, eos, gsl, yr)
        print("%d phenology is done" %yr)

still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
2007 phenology is done


In [3]:
#%%cubic+all
if __name__=='__main__':
    for yr in range(2012, 2022):
        sif=read_data(yr)
        sop1, eop1, gsl1, sop2, eop2, gsl2 = sg_cb_all(sif)
        CreatNC3(sop1, eop1, gsl1, sop2, eop2, gsl2, yr)
        print("%d phenology is done" %yr)

still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
2012 phenology is done
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
2013 phenology is done
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still alive!
still